## Using BERT for Keyword Extraction

BERT (Bidirectional Encoder Representations from Transformers) has revolutionized the way we approach natural language processing tasks, including keyword extraction. Its ability to understand context and semantics makes it particularly effective. Here’s how BERT enhances keyword extraction:

* Contextual Understanding: BERT’s bidirectional training allows it to grasp the context of words in a sentence, leading to more accurate keyword identification.
* Fine-tuning: By fine-tuning BERT on specific datasets, we can improve its performance in extracting relevant keywords tailored to particular domains.

In [1]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Read and process input text
file_path = "C:/Users/Lenovo/Documents/Rohit_AI_ML/SummariseIT/keywords.txt"
with open(file_path, "r") as file:
    text = file.read()

# Tokenize input
inputs = tokenizer(text, return_tensors='pt')

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_states = outputs.last_hidden_state

print(last_hidden_states)


tensor([[[-3.5654e-01, -2.5815e-01, -3.0364e-01,  ..., -4.4970e-01,
           3.1113e-02,  5.6097e-01],
         [-1.8669e-01, -5.7941e-01,  1.1139e-04,  ..., -4.4045e-01,
           6.6886e-01,  5.0346e-01],
         [ 1.3786e-02, -2.7492e-01,  2.9452e-01,  ..., -5.2419e-01,
          -2.6790e-02,  7.4223e-02],
         ...,
         [-6.8504e-01, -6.3286e-01,  6.4164e-01,  ..., -8.2628e-01,
          -2.5219e-01,  1.2360e-02],
         [-7.8816e-02, -1.0502e+00, -5.7024e-01,  ..., -6.2040e-02,
           1.3690e-02, -7.5901e-02],
         [ 5.3279e-01, -2.1996e-01, -1.4261e-01,  ...,  7.0739e-02,
          -8.1289e-01, -1.1717e-01]]])


In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel
import requests
from bs4 import BeautifulSoup
import re

# Load pre-trained model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Loading the dataset of keywords
datapath = r"C:\Users\Lenovo\Documents\Rohit_AI_ML\SummariseIT\dataset.csv"
df = pd.read_csv(datapath)
# Flatten the dataset to create a set of valid keywords
valid_keywords = set()
for column in df.columns:
    valid_keywords.update(df[column].dropna().str.strip().tolist())

def extract_keywords_from_tokens(text, model, tokenizer, num_keywords=5):
    # Tokenize input
    inputs = tokenizer(text, return_tensors='pt')
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_states = outputs.last_hidden_state

    # Convert token IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Get the [CLS] token's embedding
    cls_embedding = last_hidden_states[:, 0, :].squeeze()
    
    # Calculate similarity between each token embedding and the [CLS] embedding
    similarities = torch.matmul(last_hidden_states.squeeze(), cls_embedding)
    
    # Get the indices of the top-n tokens with the highest similarity
    top_indices = similarities.topk(num_keywords).indices

    # Extract the corresponding tokens, excluding [CLS] and checking if they are in valid_keywords
    keywords = [tokens[i] for i in top_indices if tokens[i] != '[CLS]' and tokens[i] in valid_keywords]
    
    return keywords

# Read and process input text
file_path = "C:/Users/Lenovo/Documents/Rohit_AI_ML/SummariseIT/keywords.txt"
with open(file_path, "r") as file:
    text = file.read()

# Use the function to extract keywords
keywords = extract_keywords_from_tokens(text, model, tokenizer)

# Print extracted keywords
print("Extracted keywords:")
for idx, keyword in enumerate(keywords, start=1):
    print(f"Keyword {idx}: {keyword}")


Extracted keywords:


## Web scraping the results of keywords from the web

In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer
import requests
from bs4 import BeautifulSoup

# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BART model and tokenizer for summarization
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Loading the dataset of keywords
datapath = r"C:\Users\Lenovo\Documents\Rohit_AI_ML\SummariseIT\dataset.csv"
df = pd.read_csv(datapath)
# Flatten the dataset to create a set of valid keywords
valid_keywords = set()
for column in df.columns:
    valid_keywords.update(df[column].dropna().str.strip().tolist())

def extract_keywords_from_tokens(text, model, tokenizer, num_keywords=5):
    # Tokenize input
    inputs = tokenizer(text, return_tensors='pt')
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state

    # Convert token IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Get the [CLS] token's embedding
    cls_embedding = last_hidden_state[:, 0, :].squeeze()
    
    # Calculate similarity between each token embedding and the [CLS] embedding
    similarities = torch.matmul(last_hidden_state.squeeze(), cls_embedding)
    
    # Get the indices of the top-n tokens with the highest similarity
    top_indices = similarities.topk(num_keywords).indices

    # Extract the corresponding tokens, excluding [CLS] and checking if they are in valid_keywords
    keywords = [tokens[i] for i in top_indices if tokens[i] != '[CLS]' and tokens[i] in valid_keywords]
    
    return keywords

# Read and process input text
file_path = "C:/Users/Lenovo/Documents/Rohit_AI_ML/SummariseIT/keywords.txt"
with open(file_path, "r") as file:
    text = file.read()

# Use the function to extract keywords
keywords = extract_keywords_from_tokens(text, bert_model, bert_tokenizer)

# Print extracted keywords
print("Extracted keywords:")
for idx, keyword in enumerate(keywords, start=1):
    print(f"Keyword {idx}: {keyword}")

    # Search the web for the keyword on Wikipedia
    search_url = f"https://en.wikipedia.org/wiki/{keyword}"
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(search_url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the relevant information from the search result
    paragraphs = soup.find_all("p")
    extracted_text = ""
    for paragraph in paragraphs:
        extracted_text += paragraph.get_text() + " "
    extracted_text = extracted_text.strip()

    # Generate a summary using BART
    def generate_summary(text, model, tokenizer):
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
        summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary

    # Generate summary
    summary = generate_summary(extracted_text, bart_model, bart_tokenizer)
    print(f"Summary of {keyword}:")
    print(summary)


Extracted keywords:
Keyword 1: accountability
Summary of accountability:
Accountability is the acknowledgment of and assumption of responsibility for actions, products, decisions, and policies. It has been central to discussions related to problems in the public sector, nonprofit, private (corporate), and individual contexts. In governance, accountability has expanded beyond the basic definition of "being called to account for one's actions"
Keyword 2: meeting
Summary of meeting:
A meeting is when two or more people come together to discuss one or more topics, often in a formal or business setting. Meetings can be used as form of group decision-making. The term "meeting" may refer to a lecture (one presentation), seminar (typically several presentations, small audience, one day) or congress.


## This code uses pretrained data of the BERT to generate result so giving gibrish and non meaningful data

In [8]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertModel, BartForConditionalGeneration, BartTokenizer

# Load pre-trained BERT model and tokenizer
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Load pre-trained BART model and tokenizer for summarization
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Loading the dataset of keywords
datapath = r"C:\Users\Lenovo\Documents\Rohit_AI_ML\SummariseIT\dataset.csv"
df = pd.read_csv(datapath)
# Flatten the dataset to create a set of valid keywords
valid_keywords = set()
for column in df.columns:
    valid_keywords.update(df[column].dropna().str.strip().tolist())

def extract_keywords_from_tokens(text, model, tokenizer, num_keywords=5):
    # Tokenize input
    inputs = tokenizer(text, return_tensors='pt')
    
    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        last_hidden_state = outputs.last_hidden_state

    # Convert token IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Get the [CLS] token's embedding
    cls_embedding = last_hidden_state[:, 0, :].squeeze()
    
    # Calculate similarity between each token embedding and the [CLS] embedding
    similarities = torch.matmul(last_hidden_state.squeeze(), cls_embedding)
    
    # Get the indices of the top-n tokens with the highest similarity
    top_indices = similarities.topk(num_keywords).indices

    # Extract the corresponding tokens, excluding [CLS] and checking if they are in valid_keywords
    keywords = [tokens[i] for i in top_indices if tokens[i] != '[CLS]' and tokens[i] in valid_keywords]
    
    return keywords

def generate_summary(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=150, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Read and process input text
file_path = "C:/Users/Lenovo/Documents/Rohit_AI_ML/SummariseIT/keywords.txt"
with open(file_path, "r") as file:
    text = file.read()

# Use the function to extract keywords
keywords = extract_keywords_from_tokens(text, bert_model, bert_tokenizer)

# Print extracted keywords
print("Extracted keywords:")
for idx, keyword in enumerate(keywords, start=1):
    print(f"Keyword {idx}: {keyword}")

    # Generate summary using the extracted keywords
    summary = generate_summary(f"what is {keyword}", bart_model, bart_tokenizer)
    print(f"Summary of {keyword}:")
    print(summary)

Extracted keywords:
Keyword 1: software
Summary of software:
what is software and how does it work? We look at some of the key features of software. What do you think? Let us know in the comments below. Back to Mail Online home. back to the page you came from."What is software?" is a weekly, interactive look at software.
